In [1]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
plt.style.use("seaborn")

-------------------------------------------------------------------------------------

# SMA backtesting code:

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
plt.style.use("seaborn")

In [41]:
api = tpqoa.tpqoa("oanda.cfg")

In [42]:
api.get_instruments()

[('AUD/CAD', 'AUD_CAD'),
 ('AUD/CHF', 'AUD_CHF'),
 ('AUD/HKD', 'AUD_HKD'),
 ('AUD/JPY', 'AUD_JPY'),
 ('AUD/NZD', 'AUD_NZD'),
 ('AUD/SGD', 'AUD_SGD'),
 ('AUD/USD', 'AUD_USD'),
 ('CAD/CHF', 'CAD_CHF'),
 ('CAD/HKD', 'CAD_HKD'),
 ('CAD/JPY', 'CAD_JPY'),
 ('CAD/SGD', 'CAD_SGD'),
 ('CHF/HKD', 'CHF_HKD'),
 ('CHF/JPY', 'CHF_JPY'),
 ('CHF/ZAR', 'CHF_ZAR'),
 ('EUR/AUD', 'EUR_AUD'),
 ('EUR/CAD', 'EUR_CAD'),
 ('EUR/CHF', 'EUR_CHF'),
 ('EUR/CZK', 'EUR_CZK'),
 ('EUR/DKK', 'EUR_DKK'),
 ('EUR/GBP', 'EUR_GBP'),
 ('EUR/HKD', 'EUR_HKD'),
 ('EUR/HUF', 'EUR_HUF'),
 ('EUR/JPY', 'EUR_JPY'),
 ('EUR/NOK', 'EUR_NOK'),
 ('EUR/NZD', 'EUR_NZD'),
 ('EUR/PLN', 'EUR_PLN'),
 ('EUR/SEK', 'EUR_SEK'),
 ('EUR/SGD', 'EUR_SGD'),
 ('EUR/TRY', 'EUR_TRY'),
 ('EUR/USD', 'EUR_USD'),
 ('EUR/ZAR', 'EUR_ZAR'),
 ('GBP/AUD', 'GBP_AUD'),
 ('GBP/CAD', 'GBP_CAD'),
 ('GBP/CHF', 'GBP_CHF'),
 ('GBP/HKD', 'GBP_HKD'),
 ('GBP/JPY', 'GBP_JPY'),
 ('GBP/NZD', 'GBP_NZD'),
 ('GBP/PLN', 'GBP_PLN'),
 ('GBP/SGD', 'GBP_SGD'),
 ('GBP/USD', 'GBP_USD'),


In [49]:
data = api.get_history(instrument = "EUR_USD", start = "2021-03-29", end = "2022-07-15",
                granularity = "M1", price = "M", localize = False)

In [50]:
type(data)
data = data.rename(columns = {'o': 'Open', 'h':'High', 'l':'Low', 'c':'Close', 'v':'Volume'})
data

,Open,High,Low,Close,volume,complete
time,,,,,,
2021-03-29 00:00:00+00:00,1.17874,1.17884,1.17870,1.17873,34,True
2021-03-29 00:01:00+00:00,1.17872,1.17873,1.17866,1.17869,31,True
2021-03-29 00:02:00+00:00,1.17868,1.17869,1.17860,1.17864,27,True
2021-03-29 00:03:00+00:00,1.17865,1.17874,1.17864,1.17871,14,True
2021-03-29 00:04:00+00:00,1.17869,1.17872,1.17867,1.17869,18,True
...,...,...,...,...,...,...
2022-07-14 23:55:00+00:00,1.00248,1.00248,1.00236,1.00242,45,True
2022-07-14 23:56:00+00:00,1.00241,1.00243,1.00240,1.00242,33,True
2022-07-14 23:57:00+00:00,1.00246,1.00254,1.00240,1.00254,25,True


In [35]:
!pip install backtesting

     |████████████████████████████████| 175 kB 808 kB/s eta 0:00:01
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173834 sha256=84a7da829cec35f43d46079fc8b5fec7585d58c6b6726725960284763bdecedc
  Stored in directory: /Users/miguelbetances/Library/Caches/pip/wheels/3f/7c/24/f8816cdb5359accfe50ebbb023baf41e98592f11528ed26ce6
Successfully built backtesting


In [37]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest


In [51]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [61]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 50
    n2 = 200
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [63]:
bt = Backtest(data, SmaCross, cash = 2_000)
bt.run()

Start                     2021-03-29 00:00...
End                       2022-07-14 23:59...
Duration                    472 days 23:59:00
Exposure Time [%]                   99.942899
Equity Final [$]                   1802.55956
Equity Peak [$]                    2004.82248
Return [%]                          -9.872022
Buy & Hold Return [%]              -14.937263
Return (Ann.) [%]                   -6.129853
Volatility (Ann.) [%]                6.352855
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.862516
Avg. Drawdown [%]                    -0.40282
Max. Drawdown Duration      464 days 17:44:00
Avg. Drawdown Duration       13 days 12:12:00
# Trades                                 3123
Win Rate [%]                         31.82837
Best Trade [%]                         1.7522
Worst Trade [%]                     -0.895316
Avg. Trade [%]                    

In [65]:
%%time

stats = bt.optimize(n1=range(20, 50, 5),
                    n2=range(50, 200, 50),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/Users/miguelbetances/opt/anaconda3/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 2min 57s, sys: 6.73 s, total: 3min 3s
Wall time: 3min 12s


Start                     2021-03-29 00:00...
End                       2022-07-14 23:59...
Duration                    472 days 23:59:00
Exposure Time [%]                   99.949852
Equity Final [$]                   1884.90287
Equity Peak [$]                     2002.6989
Return [%]                          -5.754856
Buy & Hold Return [%]              -14.937263
Return (Ann.) [%]                   -3.412736
Volatility (Ann.) [%]                6.036199
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.059445
Avg. Drawdown [%]                   -0.431574
Max. Drawdown Duration      472 days 16:03:00
Avg. Drawdown Duration       19 days 16:50:00
# Trades                                 4488
Win Rate [%]                         30.97148
Best Trade [%]                       1.757479
Worst Trade [%]                     -0.912483
Avg. Trade [%]                    

In [66]:
stats._strategy

<Strategy SmaCross(n1=30,n2=150)>

In [6]:
#api.stream_data("EUR_USD", stop = 20) 

In [33]:
# class SMABacktester():
#     def __init__(self, symbol, SMA_S, SMA_L, start, end):
#         self.symbol = symbol
#         self.SMA_S = SMA_S
#         self.SMA_L = SMA_L
#         self.start = start
#         self.end = end
#         self.results = None
#         self.get_data()
        
#     def get_data(self):
#         raw = api.get_history(instrument = "EUR_USD", start = "2021-03-29", end = "2021-04-15",
#                 granularity = "M1", price = "M", localize = False)
#         raw.rename(columns={self.symbol: "c"}, inplace=True)
#         raw["returns"] = np.log(raw / raw.shift(1))
#         raw["SMA_S"] = raw["c"].rolling(self.SMA_S).mean() # add short sma
#         raw["SMA_L"] = raw["c"].rolling(self.SMA_L).mean()
#         self.data = raw
#         return raw
    
#     def set_parameters(self, SMA_S = None, SMA_L = None):
#         if SMA_S is not None:
#             self.SMA_S = SMA_S
#             self.data["SMA_S"] = self.data["c"].rolling(self.SMA_S).mean()
#         if SMA_L is not None:
#             self.SMA_L = SMA_L
#             self.data["SMA_L"] = self.data["c"].rolling(self.SMA_L).mean()
            
#     def test_strategy(self):
#         data = self.data.copy().dropna()
#         data["position"] = np.where(data["SMA_S"] > data["SMA_L"], 1, -1)
#         data["strategy"] = data["position"].shift(1) * data["returns"]
#         data.dropna(inplace=True)
#         data["creturns"] = data["returns"].cumsum().apply(np.exp)
#         data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
#         self.results = data
        
#         perf = data["cstrategy"].iloc[-1] # absolute performance
#         outperf = perf - data["creturns"].iloc[-1] # outperformance 
#         return round(perf, 6), round(outperf, 6)
    
#     def plot_results(self):
#         if self.results is None:
#             print("Run test_strategy() first.")
#         else:
#             title = "{} | SMA_S = {} | SMA_L = {}".format(self.symbol, self.SMA_S, self.SMA_L)
#             self.results[["creturns", "cstrategy"]].plot(title=title, figsize=(12, 8))

In [67]:
#tester = SMABacktester("EURUSD=X", 50, 200, "2021-03-29", "2021-04-29")

In [26]:
#tester

SMABacktester(symbol = EURUSD=X, SMA_S = 50, SMA_L = 200, start = 2021-03-29, end = 2021-04-29)

In [68]:
#tester.test_strategy()

In [20]:
tester.results

,price,returns,SMA_S,SMA_L,position,strategy,creturns,cstrategy
Date,,,,,,,,


In [ ]:
tester.plot_results()

In [ ]:
tester.set_parameters(75, 150)

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.optimize_parameters((25, 50, 1), (100, 200, 1))

In [ ]:
tester.plot_results()

In [ ]:
tester.results_overview

# Back testing and forward testing

In [ ]:
train = SMA.SMABacktester("EURUSD=X", 50, 200, "2004-01-01", "2015-12-31")

In [ ]:
train.optimize_parameters((25, 50, 1), (100, 200, 1)) # optimizing strategy & backtesting ("in-sample")

In [ ]:
train.plot_results()

In [ ]:
tester = SMA.SMABacktester("EURUSD=X", 46, 137, "2016-01-01", "2020-06-30")

In [ ]:
tester.test_strategy() # forward testing ("out-sample")

In [ ]:
tester.plot_results()

-----------------------------------------------------------------------------------------------

# Using class 

In [69]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [70]:
class SMATrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, SMA_S, SMA_L, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.SMA_S = SMA_S
        self.SMA_L = SMA_L
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA_S"] = df[self.instrument].rolling(self.SMA_S).mean()
        df["SMA_L"] = df[self.instrument].rolling(self.SMA_L).mean()
        df["position"] = np.where(df["SMA_S"] > df["SMA_L"], 1, -1)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

In [71]:
trader = SMATrader("oanda.cfg", "EUR_USD", "1min", SMA_S = 50, SMA_L = 200, units = 100000)

In [73]:
# trader.get_most_recent()
# trader.stream_data(trader.instrument, stop = 200)
# if trader.position != 0: # if we have a final open position
#     close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
#                                       suppress = True, ret = True) 
#     trader.report_trade(close_order, "GOING NEUTRAL")
#     trader.position = 0

In [ ]:
trader.data#.tail(20)

In [ ]:
trader.data.tail(30).plot(figsize = (12, 8), secondary_y = "position")
plt.show()